In [1]:
import numpy as np
import pandas as pd
import pickle
import torch
import random
from torch_geometric.data import Data

In [2]:
cancerType = "pan"

In [ ]:
#load lables of specific cancer types
if cancerType!="pan":
    df1 = pd.read_table('../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt', index_col=0, header =0, sep="\t")
    gene_index = list(df1.index.values)
    df2 = pd.read_table('../data/Specific cancer/pos-%s.txt' %cancerType, header=None, index_col=None, names=['name'], dtype='str')
    df3 = pd.read_table('../data/2187false.txt', header=None, index_col=None, names=['name'], dtype='str')

    label_list = []
    for index,gene in enumerate(gene_index):
        flag = 0
        for driver in list(df2['name'].values):
            if gene==driver:
                label_list.append(1)
                flag = 1
        for normal in list(df3['name'].values):
            if gene==normal:
                label_list.append(0)
                flag = 1
        if flag==0:
            label_list.append(-1)
    print(len(label_list))
    label = pd.DataFrame(label_list)
    label.to_csv(path_or_buf='../data/Specific cancer/string_850/label_index_%s.txt' %cancerType, sep='\t', header=False,index=False)

In [4]:
#load labels of pan-cancer
if cancerType=="pan":
    df1 = pd.read_table('../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt', index_col=0, header =0, sep="\t")
    gene_index = list(df1.index.values)
    df2 = pd.read_table('../data/796true.txt', header=None, index_col=None, names=['name'], dtype='str')
    df3 = pd.read_table('../data/2187false.txt', header=None, index_col=None, names=['name'], dtype='str')

In [ ]:
#create label sets
driver_index = []
normal_index = []
unkown_index = []
for index,gene in enumerate(gene_index):
    flag = 0
    for driver in list(df2['name'].values):
        if gene==driver:
            driver_index.append(index)
            flag = 1
    for normal in list(df3['name'].values):
        if gene==normal:
            normal_index.append(index)
            flag = 1
    if flag==0:
        unkown_index.append(index)
        
def generate_bool_list(lst):
    return [bool(x) for x in lst]

def create_y_data():
    y_list = [0]*len(gene_index)
    for index,gene in enumerate(gene_index):
        for driver in list(df2['name'].values):
            if gene==driver:
                y_list[index] = 1
    y_te = generate_bool_list(y_list)
    y = torch.tensor(np.array(y_list).reshape(len(gene_index),-1))
    y_t = torch.tensor(np.array(y_te).reshape(len(gene_index),-1))
    y_data =Data(y=y,y_te=y_t)
    return y_data

y_data = create_y_data()
torch.save(y_data, '../data/%s/string_850/Y_796_data_2.0.pkl' %cancerType)
# torch.save(y_data, '../data/Specific cancer/string_850/Y_%s_2.0.pkl' %cancerType)

In [7]:
#divide dataset for ten five-fold cross-validations
def create_k_sets():
    data = pd.read_table("../data/pan/string_850/mut_features_miRNA_sub_du_2.0.txt", index_col=0, header =0, sep="\t")
    length = len(data.values)
    index_array = np.arange(0,length,1)
    driver_chunk_size = len(driver_index) // 5
    normal_chunk_size = len(normal_index) // 5
    k_sets_list = []
    for i in range(10):
        #randomly shuffle array order
        random.shuffle(driver_index)
        random.shuffle(normal_index)
        #divide the dataset into five parts, the last part possibly containing additional elements
        k_sets_i = []
        for j in range(5):
            tr_list = [False]*length
            for suoyin1 in driver_index:
                tr_list[suoyin1] = True
            for suoyin2 in normal_index:
                tr_list[suoyin2] = True
            te_list = [False]*length
            driver_index1 = driver_chunk_size*j
            driver_index2 = driver_chunk_size*(j+1)
            normal_index1 = normal_chunk_size*j
            normal_index2 = normal_chunk_size*(j+1)
            if j<4:
                for r1 in range(driver_index1,driver_index2):
                    index3 = driver_index[r1]
                    te_list[index3] = True
                    tr_list[index3] = False
                for r2 in range(normal_index1,normal_index2):
                    index4 = normal_index[r2]
                    te_list[index4] = True
                    tr_list[index4] = False
            if j==4:
                for r1 in range(driver_index1,len(driver_index)):
                    index3 = driver_index[r1]
                    te_list[index3] = True
                    tr_list[index3] = False
                for r2 in range(normal_index1,len(normal_index)):
                    index4 = normal_index[r2]
                    te_list[index4] = True
                    tr_list[index4] = False
            
            inputs = []
            inputs.append(tr_list)
            inputs.append(te_list)  
            k_sets_i.append(inputs)
        k_sets_list.append(k_sets_i)
    k_sets = np.array(k_sets_list)
    print(k_sets.shape)
    with open('../data/%s/string_850/k_sets_796_2.0.pkl' % cancerType, 'wb') as f:
    # with open('../data/Specific cancer/string_850/k_sets_%s_2.0.pkl' % cancerType, 'wb') as f:
        pickle.dump(k_sets, f)
        
create_k_sets()